In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

#------------------
# LOAD THE DATASET
#------------------

data = pd.read_csv('./raw_data/lastfm.csv')

# Create a new dataframe without the user ids.
data_items = data.drop('user', 1)

In [2]:
raw_tr = pd.read_json("./raw_data/train.json")
# raw_val = pd.read_json("./태그, 노래 채우기/validation.json")
# raw_te = pd.read_json("./raw_data/val.json")

In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
test_df = raw_tr
raw_te_tags_mlb = MultiLabelBinarizer(sparse_output=True)
raw_te_tags = list(test_df.tags)
te_tag_df = raw_te_tags_mlb.fit_transform(raw_te_tags)
te_tag_classes = raw_te_tags_mlb.classes_

In [4]:
raw_te_songs_mlb = MultiLabelBinarizer(sparse_output=True)
raw_te_songs = list(test_df.songs)
te_songs_df = raw_te_songs_mlb.fit_transform(raw_te_songs)
te_songs_classes = raw_te_songs_mlb.classes_

In [5]:
data_items = pd.DataFrame(te_tag_df.todense(), columns=te_tag_classes)

In [6]:
#------------------------
# ITEM-ITEM CALCULATIONS
#------------------------

# As a first step we normalize the user vectors to unit vectors.

# magnitude = sqrt(x2 + y2 + z2 + ...)
magnitude = np.sqrt(np.square(data_items).sum(axis=1))
# unitvector = (x / magnitude, y / magnitude, z / magnitude, ...)
data_items = data_items.divide(magnitude, axis='index')

def calculate_similarity(data_items):
    """Calculate the column-wise cosine similarity for a sparse
    matrix. Return a new dataframe matrix with similarities.
    """
    data_sparse = sparse.csr_matrix(data_items)
    similarities = cosine_similarity(data_sparse.transpose())
    sim = pd.DataFrame(data=similarities, index= data_items.columns, columns= data_items.columns)
    return sim

# Build the similarity matrix
data_matrix = calculate_similarity(data_items)

# # Lets get the top 11 similar artists for Beyonce
# print(data_matrix.loc['beyonce'].nlargest(11))

In [9]:
data_matrix.index[4356]

'pop추천'

In [31]:
data_matrix.index[17815]

'아침식사'

In [10]:
data_matrix.index[np.argsort(data_matrix.iloc[4356].values)[::-1][:10]]

Index(['pop추천', '취향저격노래', '트렌디음악', '알앤비음악', '데일리뮤직', 'popsong', '기분좋아지는노래',
       '도입부부터좋은', '데일리', '앤마리'],
      dtype='object')

In [52]:
data_matrix.index[8749]

'느린곡'

In [43]:
np.argsort(data_matrix.iloc[1830].values)[::-1][:10]

array([ 6600,  1830, 23712,  1829,  1156, 23710, 16686, 14373,  8387,
       11886])

In [10]:
#------------------------
# USER-ITEM CALCULATIONS
#------------------------

# Construct a new dataframe with the 10 closest neighbours (most similar)
# for each artist.

data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
for i in range(0, len(data_matrix.columns)):
    data_neighbours.ix[i,:10] = data_matrix.ix[0:,i].sort_values(ascending=False)[:10].index

user = 5985
user_index = data[data.user == user].index.tolist()[0]

# Get the artists the user has played.
known_user_likes = data_items.ix[user_index]
known_user_likes = known_user_likes[known_user_likes >0].index.values

# Construct the neighbourhood from the most similar items to the
# ones our user has already liked.
most_similar_to_likes = data_neighbours.ix[known_user_likes]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))
neighbourhood = data_matrix[similar_list].ix[similar_list]

# A user vector containing only the neighbourhood items and
# the known user likes.
user_vector = data_items.ix[user_index].ix[similar_list]

# Calculate the score.
score = neighbourhood.dot(user_vector).div(neighbourhood.sum(axis=1))

# Drop the known likes.
score = score.drop(known_user_likes)

print(known_user_likes)
print(score.nlargest(20))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-902042cf624f>", line 10, in <module>
    data_neighbours.ix[i,:10] = data_matrix.ix[0:,i].sort_values(ascending=False)[:10].index
  File "/Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/pandas/core/series.py", line 3164, in sort_values
    argsorted = _try_kind_sort(arr[good])
  File "/Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/pandas/core/series.py", line 3150, in _try_kind_sort
    return arr.argsort(kind=kind)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/yoohowon/opt/anaconda3/envs/fastcampus/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    

KeyboardInterrupt: 